In [1]:
import pickle

with open('细胞系.pkl', 'rb') as file:
    cell_dict_W = pickle.load(file)
    cell_dict_columns = pickle.load(file)
    cell_dict_high_lines = pickle.load(file)

In [2]:
with open('药物对.pkl', 'rb') as file:
    drug_dict_W = pickle.load(file)
    drug_dict_columns = pickle.load(file)
    drug_dict_high_lines = pickle.load(file)

In [3]:
import numpy as np
import pandas as pd
import re,math
from sklearn import preprocessing
from scipy.stats import pearsonr
import matplotlib.pyplot as plt


data1 = pd.read_excel(r'pcbi.1006752.s002.xlsx', sheet_name="Sheet1",index_col='Drug').drop('SMILES',axis=1)
data2 = pd.read_excel(r'Label_synergy4000.xlsx')
data2_columns = data2.columns[2:]

C:\Users\11691\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [4]:
# 结构  {名字： 名字列表， 权重： 权重列表} 顺序对应
dict_molecule_cell = {}
for elem in cell_dict_high_lines:
    dict_molecule_cell[elem] = []
    dict_molecule_cell[elem].append(cell_dict_columns[elem]["max_relevance"][-5:])
    dict_molecule_cell[elem].append(cell_dict_W[elem][-5:])

In [5]:
dict_molecule_drug = {}
for elem in drug_dict_high_lines:
    dict_molecule_drug[elem] = []
    dict_molecule_drug[elem].append(drug_dict_columns[elem]["max_relevance"][-5:])
    dict_molecule_drug[elem].append(drug_dict_W[elem][-5:])

In [6]:
# 拆分索引获取每行对应的drugA和drugB
drugA = []
drugB = []
for i in dict_molecule_drug:
    drugA.append(i.split(", ")[0][1:-1])
    drugB.append(i.split(", ")[1][1:-1])

In [7]:
# 原始数据中删除药物对得分低于0.90的行
temp = []
for index,row in data2.iterrows():
    status = 0
    for i,j in zip(drugA, drugB):
        if i == row[0] and j == row[1]:
            status = 1
    if status == 1:
        temp.append(row)
data22 = pd.DataFrame(temp)

In [8]:
# 获取得分高的细胞系名称组
cells = []
for i in dict_molecule_cell:
    cells.append(i)
cells.extend(data22.columns[0:2])

In [9]:
# 从原始数据中按列取得分高的细胞系
temp = []
for column,line in data22.iteritems():
    if column in cells:
        temp.append(line)
data222 = pd.DataFrame(temp).T

In [10]:
# 筛选结果合并drugA和drugB
tptt = []
for i,j in zip(data222['DrugA'], data222['DrugB']):
    tptt.append(f"'{i}', '{j}'")
data222 = pd.DataFrame(data222.iloc[:,2:].values,index=tptt,columns=data222.columns[2:])

In [11]:
# 原始数据合并drugA和drugB
tptt = []
for i,j in zip(data2['DrugA'], data2['DrugB']):
    tptt.append(f"'{i}', '{j}'")
data3 = pd.DataFrame(data2.iloc[:,2:].values,index=tptt,columns=data2.columns[2:])
data2 = data3.copy()

In [12]:
# 预测筛选后的药物对与细胞系的协同得分
result_df = []
for i in data222.index:
    tp = []
    for k in data222.columns[0:]:
        molecule = 0
        denominator = 0
        a = dict_molecule_cell[k][0]
        b = dict_molecule_drug[i][0]
        cw = dict_molecule_cell[k][1]
        dw = dict_molecule_drug[i][1]
        for m,ccw in zip(a,cw): # 细胞系   名字  权重
            for n,ddw in zip(b,dw): # 药物对   名字   权重
                # 分子子项 = 细胞系权重 × data2对应名字的值 ×药物对权重
                temp_molecule = ccw*data2[data2.index==n][m].values[0]*ddw
                # 分母子项 = 细胞系权重 × 药物对权重
                temp_denominator = ccw*ddw
                molecule += temp_molecule
                denominator += temp_denominator
        result = molecule / denominator
        tp.append(result)
    result_df.append(tp)

In [13]:
sens=pd.DataFrame(np.array(result_df),index = data222.index, columns=data222.columns)

In [14]:
sens

,A2058_SKIN,A2780_OVARY,CAOV3_OVARY,COLO320_LARGE_INTESTINE,HT144_SKIN,HT29_LARGE_INTESTINE,KPL1_BREAST,LOVO_LARGE_INTESTINE,MDAMB436_BREAST,NCIH1650_LUNG,NCIH2122_LUNG,NCIH460_LUNG,OVCAR4_OVARY,SKMEL30_SKIN,SKMES1_LUNG,SW620_LARGE_INTESTINE,SW837_LARGE_INTESTINE,T47D_BREAST,UACC62_SKIN,VCAP_PROSTATE
"'VINORELBINE', 'SUNITINIB'",-1.076399,10.417275,-0.782493,-0.429482,0.226380,11.560243,-0.312784,10.589955,-2.760184,-2.981405,-3.955200,-0.120279,-8.666543,0.769692,-3.915337,-2.724148,9.736702,-4.983124,2.439054,2.536755
"'AZD1775', 'MK-4827'",13.770343,10.474187,15.860605,13.852221,21.310070,7.955456,12.421949,9.678834,9.994552,10.704174,9.337820,12.503310,8.116189,20.514867,10.833237,11.600130,9.564746,13.446308,20.638770,15.513219
"'VINORELBINE', 'DASATINIB'",-10.004080,0.676263,20.421895,28.839512,5.078908,20.412901,16.074256,9.237614,-9.217700,8.610023,5.339978,-22.755271,-11.362396,10.180903,0.272132,15.182537,11.161505,32.256083,0.911889,20.902278
"'VINBLASTINE', 'MK-4827'",-14.596580,-6.153216,-14.649162,-14.429750,-28.697857,-4.574362,-22.151066,-5.754856,-16.132583,-27.666513,-2.554653,-16.829086,-18.799344,-34.652554,-20.301870,-17.473017,-4.135283,-37.021386,-30.948681,-8.888815
"'VINBLASTINE', 'AZD1775'",2.939977,1.880023,6.263010,2.510739,4.225508,5.183227,11.659663,4.788699,-8.307879,7.225488,-2.866439,-10.263217,-0.346832,2.654266,5.789966,5.022560,4.186297,29.596745,4.607850,2.868221
"'MK-4827', 'DASATINIB'",28.895421,21.733615,48.642996,56.498784,51.910029,49.092453,54.049018,44.282802,23.247377,36.160866,35.041286,10.397893,16.130711,48.353236,35.383410,41.175948,40.457204,67.820989,41.127921,49.956985
"'AZD1775', 'TOPOTECAN'",3.844619,2.971195,8.945647,3.313305,6.877460,5.226995,13.533368,4.666059,-9.672675,8.196141,-0.813410,-15.644791,-4.686930,5.323489,4.765655,4.947563,5.597989,31.910264,6.457058,2.830951
"'VINORELBINE', 'AZD1775'",4.980901,3.117648,7.979898,3.563068,10.577299,9.163150,6.706604,7.988839,-4.150918,4.396226,-2.275749,-2.436370,-3.856919,9.810954,-0.769129,2.336541,7.138581,6.930107,10.546697,4.560989
"'SUNITINIB', 'TOPOTECAN'",6.593974,7.383359,3.322376,2.889797,2.310841,4.805482,1.857994,2.104697,6.126437,2.708167,9.392501,1.155682,7.823726,-0.241880,4.862653,1.580530,6.726334,4.467987,0.804566,3.910412
"'VINBLASTINE', 'DINACICLIB'",2.234457,-1.364399,-8.580399,-13.212039,-16.734740,-1.295930,-14.368729,-3.835785,-12.087824,-12.615770,-4.768593,-12.167924,-7.576987,-10.823113,-12.268284,-10.035470,-1.729075,-14.553751,-6.452099,-6.154254


In [48]:
MSE=pow(data222-sens,2).sum().sum()/(sens.shape[0]*sens.shape[1])
MSE

505.31500727857525